In [21]:
import json
import re
from urllib.request import urlopen
import json
import html2text
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pickle
import signal

In [22]:
'''
Collect all comments by the post
Only keep posts with more than 5 comments
Takes about 3 minutes to run on timan107

To get the reddit data (chosen arbitrarily), go to the data dir and run

mkdir reddit
cd reddit
wget https://files.pushshift.io/reddit/comments/RC_2009-05.bz2
bzip2 -d RC_2009-05.bz2
'''

comments_by_post = {}
with open('../data/reddit/RC_2009-05', 'r') as f:
    for line in f:
        d = json.loads(line)
        link_id = d['link_id']
        if link_id not in comments_by_post:
            comments_by_post[link_id] = {}
        d['body'] = html2text.html2text(d['body'])
        comments_by_post[link_id][d['id']] = d
        
for key in list(comments_by_post.keys()):
    if len(comments_by_post[key]) < 5:
        del comments_by_post[key]

In [23]:
def collect_ancestors(comments: dict, comment_id: str) -> list:
    '''
    For a given list of comments and a comment in the list, reconstruct a path to the top-level comment
    Returns a list of comment IDs

    This method isn't very efficient but good enough for now
    '''
    ancestors = []
    while True:
        
        if comment_id[:2] == 't3': 
            # refers to a link (top-level comment)
            # means we've reached the top of the chain
            return ancestors[::-1]

        if comment_id[:2] == 't1':
            comment_id = comment_id[3:]

        try:
            # there is an error here sometimes where the comment id is not present in the list
            # probably fine for now, but may need to address in the future
            old_comment_id = comment_id
            comment_id = comments[comment_id]['parent_id']
            ancestors.append(old_comment_id)
        except:
            return ancestors[::-1]

In [25]:
'''
Cycle through all posts and comments, find any URL mentions, and save the mention location + the comment's ancestors
'''
all_urls = []
for post_id in comments_by_post:
    for comment_id in comments_by_post[post_id]:
        
        # first check if post body contains URL
#         urls = re.findall(r'(https?://\S+)', comments_by_post[post_id][comment_id]['body'])
#         len1 = len(urls)

        # check if post body contains URL, accounts for edge case when dash is at the end of the line
        current_comment_text = comments_by_post[post_id][comment_id]['body']
        urls = re.findall(r'(https?://\S+-\n)?(?(1)([\S]*)|(https?://\S+))', current_comment_text)
        
#         len2 = len(urls)
        
        #if len1 != len2:
        #    print(current_comment_text)
                
        if urls:
            ancestors = collect_ancestors(comments_by_post[post_id], comment_id)
            
            for url in urls:
                url = "".join(list(url))

                # heuristics for parsing errors
                url = re.sub('\)', '', url)
                url = re.sub('\]', '', url)
                url = re.sub('\n', '', url)
                
                # remove non-alphnumeric characters
                url_letters = re.sub('[^0-9a-zA-Z]', '', url)
                                
                # ignore pdfs
                if 'pdf' == url_letters[-3:] or 'jpg' in url_letters[-3:] or 'png' in url_letters[-3:] or 'gif' in url_letters[-3:]:
                    continue
                
                all_urls.append({'post_id': post_id, 'comment_id': comment_id, 'url': url, 'ancestors': ancestors})

urls_with_context = [x for x in all_urls if len(x['ancestors']) > 2]

In [26]:
'''
Loops through all ancestors of a URL comment and returns the chain up to to the top comment
'''

def get_context(url_obj: dict) -> list:
    post_id = url_obj['post_id']
    context = []
    for ancestor in url_obj['ancestors']:
        context.append(comments_by_post[post_id][ancestor]['body'])
    return context

In [27]:
'''
Simple method to scrape text from URLs. Not very robust. Need to handle exceptions. YouTube links take very long
'''

class TimeoutException(Exception):
    pass

def handler(signum, frame):
    raise TimeoutException

def scrape(url: str) -> str:
    try:
        html = urlopen(url).read()
    except Exception as e:
        return ""

    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines and short lines
    text = ""
    for chunk in chunks:
        if chunk and len(chunk) > 50:
            # inspired by https://bigscience.huggingface.co/blog/building-a-tb-scale-multilingual-dataset-for-language-modeling#:~:text=Filters%2C%20tools%2C%20and%20indicators%20of%20data%20quality
            only_text = re.sub('[^ \w\*]', '', chunk)
            if len(only_text) / len(line) > 0.8:
                text += chunk

    return text

In [28]:
# explore distribution of domains (help us with parsing)
domains = {}
for url in urls_with_context:
    try:
        domain = urlparse(url['url']).netloc
    except:
        print(url['url'])
    if domain not in domains:
        domains[domain] = 0
    domains[domain] += 1

sorted_domains = sorted([(domain, domains[domain]) for domain in domains], reverse=True, key=lambda x: x[1])
print(sorted_domains[:100])

http://www.enamerique.com[(2(http://www.enamerique.net
[('en.wikipedia.org', 6012), ('www.reddit.com', 3141), ('www.youtube.com', 2591), ('www.google.com', 386), ('www.amazon.com', 340), ('www.imdb.com', 251), ('scriptures.lds.org', 217), ('news.bbc.co.uk', 166), ('www.biblegateway.com', 160), ('www.nytimes.com', 147), ('www.flickr.com', 137), ('tinyurl.com', 126), ('video.google.com', 119), ('www.guardian.co.uk', 106), ('dictionary.reference.com', 104), ('xkcd.com', 103), ('www.merriam-webster.com', 95), ('images.google.com', 92), ('lokonline.com', 76), ('lmgtfy.com', 75), ('books.google.com', 73), ('www.urbandictionary.com', 73), ('addons.mozilla.org', 64), ('maps.google.com', 56), ('www.cnn.com', 52), ('www.washingtonpost.com', 51), ('www.timesonline.co.uk', 50), ('www.msnbc.msn.com', 49), ('www.snopes.com', 48), ('pickyourhours.com', 48), ('www.theonion.com', 47), ('www.cdc.gov', 45), ('www.newegg.com', 44), ('www.yooouuutuuube.com', 44), ('code.google.com', 43), ('www.ted.com', 43

In [29]:
# make a basic training data set for bbc
# url_keywords = ['youtube.com']
url_keywords = ['bbc.co.uk', 'cnn.com', 'wikipedia.org']
training_data = []

for i, url in enumerate(urls_with_context):
    if i % 1000 == 0: print(i / len(urls_with_context))
    
    if len(training_data) == 2000:
        break

    for keyword in url_keywords:
        if keyword in url['url']:
            
#             print(url['url'])

            try:
                signal.signal(signal.SIGALRM, handler)
                signal.alarm(1) # 5 second timeout
                
                text = scrape(url['url'])
                
                signal.alarm(0) # disable alarm
                
                if text != '':
                    url['text'] = text
                    training_data.append(url)
            except:
                print(url['url'])

0.0
http://en.wikipedia.org/wiki/Koreans
http://en.wikipedia.org/wiki/Lloyd_Doggett#Abortion
http://en.wikipedia.org/wiki/Excommunication#Jehovah.27s_Witnesses
http://en.wikipedia.org/wiki/BBC#Revenue
http://en.wikipedia.org/wiki/Arab_citizens_of_Israel
http://en.wikipedia.org/wiki/Amnesty_International#Work.
http://en.wikipedia.org/wiki/Bush_v._Gore
http://en.wikipedia.org/wiki/List_of_common_misconceptions#Islam
http://en.wikipedia.org/wiki/Israel#Religion
0.0323907621546335
http://en.wikipedia.org/wiki/Copyright#History
http://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate
http://en.wikipedia.org/wiki/French_Republic
http://en.wikipedia.org/wiki/Africa#Colonialism_and_the_.22Scramble_for_Africa.22
http://en.wikipedia.org/wiki/Crime_in_the_United_States#Characteristics_of_offenders
http://en.wikipedia.org/wiki/Peckham_Library
0.064781524309267
http://en.wikipedia.org/wiki/Bladder_cancer
http://news.bbc.co.uk/1/hi/uk/3996677.stm
http://en.wikipedia.org/wiki/France
0.097172286

In [30]:
# sets up a general training data set by adding context of all comments

for example in training_data:
    # get text of comment + ancestor comments
    context = get_context(example)
    example['full_context'] = context

pickle.dump(training_data, open('../data/reddit/bbc_news_scrape_raw.pkl', 'wb'))

In [31]:
for example in training_data:
    print(example['text'], 2 * "\n", example['full_context'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
# format training data for pyserini (https://github.com/castorini/pyserini/)

pyserini_retrieval_docs = []
relevance_scores = []
for i,example in enumerate(training_data):
    doc = {"id": i, "contents": example['text']}
    pyserini_retrieval_docs.append(doc)
    relevance_score = str(i) + ' 0 ' + str(i) + ' 1'
    relevance_scores.append(relevance_score)

with open('../data/reddit/pyserini/bbc_news_pyserini.jsonl', 'w') as f:
    for doc in pyserini_retrieval_docs:
        f.write(json.dumps(doc) + '\n')

with open('../data/reddit/bbc_news_rel.txt', 'w') as f:
    for rs in relevance_scores:
        f.write(rs + '\n')